# Superdense Coding #
$$
    \newcommand{\ket}[1]{\left|#1\right\rangle}
    \newcommand{\id}{ 	𝟙 }
$$

## Introduction ##

In this Notebook, we'll briefly go over how to implement and test superdense coding in Python with QuTiP. As usual, Nielsen and Chuang is a great resource. If you don't have a copy handy, thoughy, Wikipedia has a [nice summary](https://en.wikipedia.org/wiki/Superdense_coding).

As usual, we start by importing QuTiP. For this example, that's the only package we will need--- in particular, we won't be manipulating arrays, so we won't need NumPy.

In [1]:
import qutip as qt

For convienence, though, we'll grab one function out of IPython itself: ``display``. This lets us pretty-print objects even inside loops and such.

In [2]:
from IPython.display import display

## Initial State ##

The superdense coding protocol starts off with a two-qubit state $\ket{\beta_{00}} = (\ket{00} + \ket{11}) / \sqrt{2}$. We can construct this state in a few different ways. Let's start with the most direct way, using a Qobj initialized from a Python list:

In [3]:
beta_00 = qt.Qobj([[1], [0], [0], [1]], dims=[[2, 2], [1, 1]]).unit()
beta_00

Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.70710678]]

Note that we have used ``unit()`` here to automatically normalize the vector, so that we didn't need to worry with the factor of $1 / \sqrt{2}$. We did have this pain of specifying ``dims`` manually, however, since otherwise QuTiP wouldn't know that we wanted a two-qubit state and not a qu4it state. Next, we'll rely on ``qt.tensor`` to handle the metadata. This has the nice bonus of matching the mathematical notation above more closely.

In [4]:
beta_00 = (
    qt.tensor(qt.basis(2, 0), qt.basis(2, 0)) +
    qt.tensor(qt.basis(2, 1), qt.basis(2, 1))
).unit()
beta_00

Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.70710678]]

Better, but now it's pretty verbose. To make it all fit in a readable fashion we had to take advantage of how Python handles newlines: if you open a pair of parentheses, the line isn't over until it's closed. That way we can "wrap" one line across several to make it easier to read.

Though it's a bit more advanced, but we can use a *generator expression* to simplify things a bit.

In [5]:
beta_00 = sum(qt.tensor([qt.basis(2, idx)] * 2) for idx in range(2)).unit()
beta_00

Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.70710678]]

This has the advantage of working for more than just qubits with a slight generalization, but is probably overkill for what we need right now.

The next way we'll consider will be useful for writing the decoder later, and that's to use a Hadamard and a CNOT unitary to make the Bell state from the computational basis state $\ket{00}$. That is, we use that $\ket{\beta_{00}} = \text{CNOT} (H \otimes \id) \ket{00}$.

In [6]:
beta_00 = qt.cnot(2, 0, 1) * qt.tensor(qt.hadamard_transform(1), qt.qeye(2)) * (qt.tensor([qt.basis(2, 0)] * 2))
beta_00

Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.70710678]]

Finally, we can rely on QuTiP having a ``bell_state`` function built in. (For the curious, they implement this by using ``tensor`` and ``basis``--- [take a look](https://github.com/qutip/qutip/blob/master/qutip/states.py#L1098) if you like!)

In [7]:
beta_00 = qt.bell_state()
beta_00

Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.70710678]]

## Encoding ##

Next, we need Alice to encode her two classical bits (cbits) into the state she shares with Bob. Let's call them $i$ and $j$. As is usual for superdense coding, we suppose that Alice and Bob start far apart, such that Alice can only do things on her half of their state. That is, we need to evolve under a unitary $U = U_A(i, j) \otimes \id_B$, where $U_A(i, j)$ is a unitary on Alice's side and $\id_B$ represents that what Bob does can't depend on $i$ and $j$, since he doesn't know those yet. That is, he might as well just do nothing, represented by an identity operator.

In particular, Alice's gate is given by 
\begin{equation}
     U_A(i, j) = \begin{cases}
         \id & (0, 0) \\
         \sigma_x & (0, 1) \\
         \sigma_z & (1, 0) \\
         i \sigma_y & (1, 1)
    \end{cases}.
\end{equation}
Let's look at how we could implement this as a function.

In [8]:
def alice_encode(i, j):
    if i == 0 and j == 0:
        return qt.qeye(2)
    elif i == 0 and j == 1:
        return qt.sigmax()
    elif i == 1 and j == 0:
        return qt.sigmaz()
    elif i == 1 and j == 1:
        return 1j * qt.sigmay()

In [9]:
alice_encode(1, 1)

Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = False
Qobj data =
[[ 0.  1.]
 [-1.  0.]]

Phew, that was a lot of writing thanks to all those ``if`` statements! All we really wanted to do was to map the inputs $i$ and $j$ to an output we knew ahead of time. Thankfully, Python allows us to use a *dictionary* to express the same idea:

In [10]:
def alice_encode(i, j):
    return {
        (0, 0): qt.qeye(2),
        (0, 1): qt.sigmax(),
        (1, 0): qt.sigmaz(),
        (1, 1): qt.sigmay()
    }[(i, j)]

Not that much shorter, but a lot less typing and it follows the math more closely. We can do still better, though, by exploiting that $i \sigma_y = \sigma_z \sigma_x$:

In [11]:
1j * qt.sigmay() == qt.sigmaz() * qt.sigmax()

True

In [12]:
def alice_encode(i, j):
    return qt.sigmaz() ** i * qt.sigmax() ** j

In [13]:
alice_encode(1, 1)

Quantum object: dims = [[2], [2]], shape = [2, 2], type = oper, isherm = False
Qobj data =
[[ 0.  1.]
 [-1.  0.]]

Now that we have a function that we like for $U_A$, let's use it to define the unitary for both Alice and Bob.

In [14]:
def both_encode(i, j):
    return qt.tensor(alice_encode(i, j), qt.qeye(2))

Let's see what happens to their Bell state for each of Alice's inputs.

In [15]:
for i in range(2):
    for j in range(2):
        print "Alice's bits: {}, {}".format(i, j)
        display(both_encode(i, j) * beta_00)

Alice's bits: 0, 0


Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.70710678]]

Alice's bits: 0, 1


Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.        ]
 [ 0.70710678]
 [ 0.70710678]
 [ 0.        ]]

Alice's bits: 1, 0


Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [-0.70710678]]

Alice's bits: 1, 1


Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.        ]
 [ 0.70710678]
 [-0.70710678]
 [ 0.        ]]

## Decoding

Next, Alice sends Bob her half of the state, so that Bob can now do a unitary on the entire state. To decode and figure out which bits Alice sent, let's undo how we made the state, using a Hadamard and a CNOT gate.

In [16]:
bob_decode = qt.tensor(qt.hadamard_transform(1), qt.qeye(2)) * qt.cnot(2, 0, 1)
bob_decode

Quantum object: dims = [[2, 2], [2, 2]], shape = [4, 4], type = oper, isherm = False
Qobj data =
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.70710678  0.          0.         -0.70710678]
 [ 0.          0.70710678 -0.70710678  0.        ]]

Does it work? Let's look what it does for each state:

In [17]:
for i in range(2):
    for j in range(2):
        print "Alice's bits: {}, {}".format(i, j)
        display(bob_decode * both_encode(i, j) * beta_00)

Alice's bits: 0, 0


Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 1.]
 [ 0.]
 [ 0.]
 [ 0.]]

Alice's bits: 0, 1


Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.]
 [ 1.]
 [ 0.]
 [ 0.]]

Alice's bits: 1, 0


Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.]
 [ 0.]
 [ 1.]
 [ 0.]]

Alice's bits: 1, 1


Quantum object: dims = [[2, 2], [1, 1]], shape = [4, 1], type = ket
Qobj data =
[[ 0.]
 [ 0.]
 [ 0.]
 [ 1.]]

Perfect! One last thing, though, we had to manually look at each of these to see that we got the right answer. We know what the answer should be in this case, though, so we can write a *test case* to check. A test case is a function that raises an error if another piece of code is malfunctioning, that way you can sometimes tell if you introduce new bugs when you change things.

In [18]:
def test_superdense():
    for i in range(2):
        for j in range(2):
            expected = qt.tensor(qt.basis(2, i), qt.basis(2, j))
            assert expected == bob_decode * both_encode(i, j) * beta_00

In [19]:
test_superdense()

The ``assert`` keyword raises an error if you give it something ``False``, and otherwise does nothing. Since the function returned without an error, that checks that we did it correctly.